## Problem Statement
Dalam pengerjaan koreksi satelit, perlu dilakukan windowing berdasarkan kategori hujan, sehingga dibutuhkan tahapan untuk mengubah data hujan berdasarkan kategori hujan dari BMKG. Adapun dalam pengerjaan koreksi data satelit, dibutuhkan data hujan observasi yang berperan sebagai __corrector__ dan adapula data satelit yang ingin dikoreksi di daerah irigasi. Biasanya data satelit digunakan untuk area yang tidak memiliki data ch obs sehingga akan dilakukan koreksi CH satelit menggunakan data CH dari stasiun observasi di lokasi terdekat.

## Objective
- Script ini digunakan untuk mengklasifikasikan data hujan ke dalam kategori ringan, sedang, lebat, dan sangat lebat.

In [1]:
import pandas as pd
import numpy as np
import openpyxl
import glob
from datetime import datetime, timedelta

In [2]:
# read satellite data
path_satellite =r'D:\Happy\satellite_correction\data_satellite'
rainfall_files = glob.glob(path_satellite + "/*.xlsx")

In [3]:
rainfall_sat=pd.read_excel(path_satellite+'/Hasil_GSMAP_StasiunJeneberang.xlsx')
rainfall_obs=pd.read_excel('D:/Happy/satellite_correction/uji_data_hujan_obs/rainfall_QC.xlsx').drop(['Unnamed: 0'], axis=1)
rainfall_sat

,date,PCH STASIUN MANGEMPANG,PCH STASIUN LENGKESE,PCH STASIUN LIMBUNGA,PCH STASIUN MALINO,PCH STASIUN PANAIKANG,PCH STASIUN POS1,PCH STASIUN KD1,PCH STASIUN BILI-BILI,PCH STASIUN KAMPILI,PCH STASIUN MACINI SOMBALA
0,2000-03-01,0.030149,0.041025,0.035131,0.042305,0.042305,0.042305,0.037572,0.026950,0.030328,0.045610
1,2000-03-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2000-03-03,0.037995,0.066958,0.038232,0.144621,0.144621,0.144621,0.070834,0.057893,0.048813,0.021543
3,2000-03-04,0.003088,0.000000,0.002961,0.000000,0.000000,0.000000,0.000000,0.001286,0.001522,0.000000
4,2000-03-05,0.001816,0.008126,0.004855,0.011265,0.011265,0.011265,0.002208,0.001738,0.002306,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
8487,2023-05-27,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8488,2023-05-28,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8489,2023-05-29,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8490,2023-05-30,0.289392,0.203689,0.328786,0.333063,0.333063,0.333063,0.349100,0.222015,0.203741,0.123013


In [4]:
start_date = '2014-01-01'
end_date = '2018-12-31'

# Select DataFrame rows between two dates
mask = (rainfall_sat['date'] > start_date) & (rainfall_sat['date'] <= end_date)
rainfall_sat = rainfall_sat.loc[mask].drop(['PCH STASIUN MACINI SOMBALA'], axis=1).reset_index(drop=True)
rainfall_sat.head()

,date,PCH STASIUN MANGEMPANG,PCH STASIUN LENGKESE,PCH STASIUN LIMBUNGA,PCH STASIUN MALINO,PCH STASIUN PANAIKANG,PCH STASIUN POS1,PCH STASIUN KD1,PCH STASIUN BILI-BILI,PCH STASIUN KAMPILI
0,2014-01-02,0.273670,0.273395,0.271532,0.170238,0.170238,0.170238,0.135333,0.148753,0.106239
1,2014-01-03,1.389740,2.011782,1.791551,1.734499,1.734499,1.734499,1.634039,1.596254,1.397939
2,2014-01-04,0.174436,0.167657,0.167565,0.051916,0.051916,0.051916,0.054207,0.126945,0.156666
3,2014-01-05,0.041082,0.694548,0.349712,0.456225,0.456225,0.456225,0.237710,0.000000,0.052521
4,2014-01-06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [5]:
# read excel satellite data
 
#Klasifikasi hujan
# loop berdasarkan urutan di excel stasiun gpm per gage tiap model (per nama stasiun, agar yg dipanggil bukan indeks kolom)
for i in (rainfall_sat.columns[1:]):
    gage=rainfall_sat[i]

    #create dataframe with obs data
    sat_tmp={"date":rainfall_sat.loc[:,"date"],'satellite':gage}
    globals()["gage_"+i[12:]]=pd.DataFrame(sat_tmp)

    #klasifikasi obs
    conditions = [
        (gage >= 0.) & (gage< 20),
        (gage >= 20) & (gage < 50),
        (gage >= 50) & (gage < 100),
        (gage >= 100)
        ]

    # create a list of the values we want to assign for each condition
    values = ['Ringan','Sedang','Lebat','Sangat_Lebat']

    # create a new column and use np.select to assign values to it using our lists as arguments
    globals()["gage_"+i[12:]]['Klasifikasi'] = np.select(conditions, values)
    globals()["gage_"+i[12:]][i] = rainfall_obs[i]

    print ('Done: '+i)
    
#Split gpm table based on rainfall classification for each ch area
for k in (rainfall_sat.columns[1:]) :
    for value in (values) :
        globals()[value+"_gage"+k[12:]] = \
        globals()["gage_"+k[12:]][globals()["gage_"+k[12:]]['Klasifikasi']==value].reset_index(drop=True)

#save into excel per model dan klasifikasi
for value in values:
    with pd.ExcelWriter('D:/Happy/satellite_correction/klasifikasi_hujan/classified_GSMAP/acuan_koreksi/Hasil_GSMAP_StasiunJeneberang_classified_'
                        +value+'.xlsx') as writer: 
        for gage_name in (rainfall_sat.columns[1:]):
            globals()[value+"_gage"+gage_name[12:]].set_index('date').to_excel(writer, sheet_name=gage_name)
    
    

Done: PCH STASIUN MANGEMPANG
Done: PCH STASIUN LENGKESE
Done: PCH STASIUN LIMBUNGA
Done: PCH STASIUN MALINO
Done: PCH STASIUN PANAIKANG
Done: PCH STASIUN POS1
Done: PCH STASIUN KD1
Done: PCH STASIUN BILI-BILI
Done: PCH STASIUN KAMPILI


In [6]:
globals()["gage_"+i[12:]]

,date,satellite,Klasifikasi,PCH STASIUN KAMPILI
0,2014-01-02,0.106239,Ringan,1.0
1,2014-01-03,1.397939,Ringan,2.0
2,2014-01-04,0.156666,Ringan,5.0
3,2014-01-05,0.052521,Ringan,0.0
4,2014-01-06,0.000000,Ringan,1.0
...,...,...,...,...
1820,2018-12-27,0.168267,Ringan,4.0
1821,2018-12-28,0.000000,Ringan,17.0
1822,2018-12-29,0.000000,Ringan,4.0
1823,2018-12-30,0.000000,Ringan,0.0


## Rainfall Satellite for Model/Water Allocation

In [7]:
path_sat_m=r'D:\Happy\satellite_correction\data_satellite\Bilibili_Rainfall'
rainfall_sat_m=pd.read_excel(path_sat_m+'/Hasil_Kotak2_GSMAP_JeneberangIndeks.xlsx')
rainfall_sat_m

,date,0,1,2,3,4,5,6,7,8,...,22,23,24,25,26,27,28,29,30,31
0,2000-03-01,0.045065,0.045610,0.047028,0.046331,0.055258,0.070912,0.104912,0.124258,0.032565,...,0.041025,0.045692,0.021543,0.017693,0.023779,0.026909,0.037263,0.051096,0.059471,0.056581
1,2000-03-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2000-03-03,0.000000,0.021543,0.049667,0.110059,0.108090,0.098651,0.161191,0.197145,0.015190,...,0.066958,0.218831,0.042569,0.037854,0.034546,0.041370,0.053991,0.022329,0.047191,0.109371
3,2000-03-04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.085031,0.059497,0.023973,0.002059,0.006426,0.007270,0.006820,0.080345,0.178517
4,2000-03-05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.008126,0.000000,0.005762,0.004954,0.002430,0.001898,0.004998,0.002542,0.000000,0.002471
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8487,2023-05-27,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8488,2023-05-28,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8489,2023-05-29,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8490,2023-05-30,0.144815,0.123013,0.197438,0.289563,0.247136,0.310275,0.397710,0.374545,0.145259,...,0.203689,0.218594,0.259550,0.346574,0.381352,0.312960,0.273931,0.259198,0.290468,0.262426


In [8]:
rainfall_sat_m.columns[1:]

Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31],
      dtype='object')

In [9]:
# read excel satellite data
 
#Klasifikasi hujan
# loop berdasarkan urutan di excel stasiun sat per gage tiap model
for i in (rainfall_sat_m.columns[1:]):
    gage=rainfall_sat_m[i]

    #create dataframe with sat data
    sat_tmp={"date":rainfall_sat_m.loc[:,"date"],'gage':gage}
    globals()["gage_"+str(i)]=pd.DataFrame(sat_tmp)

    #klasifikasi sat
    conditions = [
        (gage >= 0.) & (gage< 20),
        (gage >= 20) & (gage < 50),
        (gage >= 50) & (gage < 100),
        (gage >= 100)
        ]

    # create a list of the values we want to assign for each condition
    values = ['Ringan','Sedang','Lebat','Sangat_Lebat']

    # create a new column and use np.select to assign values to it using our lists as arguments
    globals()["gage_"+str(i)]['Klasifikasi'] = np.select(conditions, values)

    print ('Done: '+str(i))
    
#Split gpm table based on rainfall classification for each ch area
for k in (rainfall_sat_m.columns[1:]) :
    for value in (values) :
        globals()[value+"_gage"+str(k)] = \
        globals()["gage_"+str(k)][globals()["gage_"+str(k)]['Klasifikasi']==value].reset_index(drop=True)

#save into excel per model dan klasifikasi
for value in values:
    with pd.ExcelWriter(path_sat_m+'/classified_Bilibili_Rainfall/box_shape_data/Bilibili_sat_box_GSMAP_classified_'
                        +value+'.xlsx') as writer: 
        for gage_name in (rainfall_sat_m.columns[1:]):
            globals()[value+"_gage"+str(gage_name)].to_excel(writer, sheet_name=str(gage_name))
    
    

Done: 0
Done: 1
Done: 2
Done: 3
Done: 4
Done: 5
Done: 6
Done: 7
Done: 8
Done: 9
Done: 10
Done: 11
Done: 12
Done: 13
Done: 14
Done: 15
Done: 16
Done: 17
Done: 18
Done: 19
Done: 20
Done: 21
Done: 22
Done: 23
Done: 24
Done: 25
Done: 26
Done: 27
Done: 28
Done: 29
Done: 30
Done: 31


In [10]:
globals()['Ringan'+"_gage"+str(gage_name)]

,date,gage,Klasifikasi
0,1981-01-01,5.369995,Ringan
1,1981-01-02,10.266672,Ringan
2,1981-01-03,2.977593,Ringan
3,1981-01-04,5.146244,Ringan
4,1981-01-05,3.734900,Ringan
...,...,...,...
13370,2023-05-26,0.000000,Ringan
13371,2023-05-28,0.000000,Ringan
13372,2023-05-29,0.000000,Ringan
13373,2023-05-30,0.000000,Ringan
